In [11]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
from data_utils import * 
from Trainer import *



def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):

    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    print(X_train.shape)

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel
    #print(mean_pixel.shape)
    return X_train, y_train, X_val, y_val, X_test, y_test




X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

(49000, 32, 32, 3)
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [7]:
class CustomConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2,channel_3, num_classes):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        #initializer = tf.initializers.VarianceScaling(scale=2.0)
        initializer = tf.keras.initializers.GlorotNormal()
        self.conv1 = tf.keras.layers.Conv2D(filters = channel_1, kernel_size = [3, 3],
                                           strides = 1, padding='same', kernel_initializer=initializer)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu1 = tf.keras.layers.ReLU()
        self.drop1 = tf.keras.layers.Dropout(rate = 0.5)
        self.max_pool1 = tf.keras.layers.MaxPool2D(pool_size = [2, 2], strides = [2, 2], padding = 'same')
        self.conv2 = tf.keras.layers.Conv2D(filters = channel_2, kernel_size = [3, 3],
                                           strides = 1, padding='same', kernel_initializer=initializer)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.relu2 = tf.keras.layers.ReLU()
        self.drop2 = tf.keras.layers.Dropout(rate = 0.5)
        self.max_pool2 = tf.keras.layers.MaxPool2D(pool_size = [2, 2], strides = [2, 2], padding = 'same')
        
        self.conv3 = tf.keras.layers.Conv2D(filters = channel_3, kernel_size = [3, 3],
                                           strides = 1, padding='same', kernel_initializer=initializer)
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.relu3 = tf.keras.layers.ReLU()
        self.drop3 = tf.keras.layers.Dropout(rate = 0.5)
        self.max_pool3 = tf.keras.layers.AveragePooling2D(pool_size = [5, 5], strides = [1, 1], padding = 'same')
        
        self.flatten = tf.keras.layers.Flatten()
        self.fc3 = tf.keras.layers.Dense(100, kernel_initializer=initializer)
        self.fc4 = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)
        

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        x = input_tensor
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.max_pool1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.max_pool2(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.drop3(x)
        x = self.max_pool3(x)
        
        x = self.flatten(x)
        x = self.fc3(x)
        x = self.fc4(x)
        
        #channel_1, channel_2, num_classes = 16, 8, 10

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        
        return x

#device = '/device:GPU:0'   # Change this to a CPU/GPU as you wish!
device = '/cpu:0'        # Change this to a CPU/GPU as you wish!
print_every = 700
num_epochs = 10
channel_1, channel_2, channel_3, num_classes = 32, 64, 128, 10
model = CustomConvNet(channel_1, channel_2, channel_3, num_classes)

def model_init_fn():
    return CustomConvNet(channel_1, channel_2, channel_3, num_classes)

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 
train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

NameError: name 'tf' is not defined